In [7]:
#1
import numpy as np
from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

df = pd.read_csv("ZIONO_weekly_return_volatility.csv")
df.label = df.label.fillna(method='ffill')
df_21 = df[df.Year==2021]
df_22 = df[df.Year==2022]

df_22.head(10)


df_22.head() # mean


,Year,Week_Number,mean_return,volatility,label
54,2022,1,-0.2012,0.842329,red
55,2022,2,0.1002,0.604448,green
56,2022,3,0.1525,0.591757,green
57,2022,4,-0.3576,1.847664,red
58,2022,5,-0.1206,0.762250,red


In [8]:
# 1 
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

x_21 = df_21[["mean_return"]].values
le = LabelEncoder()
y_21 = le.fit_transform(df_21["label"].values)
y_22 = le.transform(df_22["label"].values)

df_train = pd.DataFrame({'x': df_21["mean_return"].values, 'y': y_21})
df_train.head()

,x,y
0,-0.2586,0
1,0.0776,1
2,0.9710,0
3,-0.5042,1
4,0.1458,0


In [9]:
from scipy import stats
# assume data is in array x
df_train_red = df_train[df_train.y == 1]
print(df_train_red.shape)
df_red, location_red , scale_red = stats.t.fit(df_train_red.x.values)


df_red , location_red , scale_red

(30, 2)


(2.893414966233393, -0.15939134804801525, 0.13000994165478075)

In [10]:
df_train_green = df_train[df_train.y == 0]
print(df_train_green.shape)

df_green , location_green , scale_green = stats .t.fit(df_train_green.x.values)
df_green , location_green , scale_green

(22, 2)


(2.0959833273079256, 0.21078840158371498, 0.20558682732599712)

In [11]:
# NB_classifier = GaussianNB ().fit ( x_21 , y_21 )

def get_red_or_green(z, _df):
#     print(z)
    red_prop = stats.t.pdf(z,df = _df, loc = location_red , scale = scale_red )
    green_prop = stats.t.pdf(z,df = _df, loc= location_green , scale = scale_green )
    p_red = red_prop * 30 / 52
    p_green = green_prop * 22 / 52
    normalized_red = p_red / (p_red + p_green)
    normalized_green = p_green / (p_red + p_green)
    return 1 if normalized_red > normalized_green else 0
    
 

In [12]:

def get_acc_for_single_freedom(d_freedom):
    print("df=", d_freedom)
    df_22['predict'] = df_22.mean_return.apply(lambda x: get_red_or_green(x, d_freedom))
    y_22 = le.transform(df_22["label"].values)


    ret_df = pd.DataFrame({"predict": df_22['predict'].values, "ground_label": y_22})

    acc = accuracy_score(y_22, ret_df['predict'])
    print("acc=", acc)
    
    print("confusion=\n", confusion_matrix(y_22, ret_df['predict']))
    
    tn, fp, fn, tp = confusion_matrix(y_22,  ret_df['predict']).ravel()
    tn, fp, fn, tp
    tnr = tn / (tn + fp)
    tpr = tp / (tp + fn)

    print("true negative rate=", tnr)
    print(" true positive rate =", tpr)
    print("=====================================================")
    print("")
    print("")
    return acc, ret_df, y_22

result_list = []
for deg in [0.5, 1.0, 1.5]:
    result_list.append(get_acc_for_single_freedom(deg))
    

# below is result for 1, 2, 3
# df = 0.5; 1.0; 1.5

df= 0.5
acc= 0.8846153846153846
confusion=
 [[22  5]
 [ 1 24]]
true negative rate= 0.8148148148148148
 true positive rate = 0.96


df= 1.0
acc= 0.9038461538461539
confusion=
 [[23  4]
 [ 1 24]]
true negative rate= 0.8518518518518519
 true positive rate = 0.96


df= 1.5
acc= 0.9038461538461539
confusion=
 [[23  4]
 [ 1 24]]
true negative rate= 0.8518518518518519
 true positive rate = 0.96




d:\ProgramData\Anaconda3\envs\sklearn\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
d:\ProgramData\Anaconda3\envs\sklearn\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
d:\ProgramData\Anaconda3\envs\sklearn\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWa

In [13]:
df_15 = result_list[2][1]
df_15.head()

,predict,ground_label
0,1,1
1,0,0
2,0,0
3,1,1
4,1,1


In [14]:
# 4
# best df = 1.5

# acc= 0.9038461538461539 is much bether than normal Naive Bayesian, which is 0.75

In [15]:
#  5
df_detail = pd.read_csv("ZIONO_weekly_return_detailed.csv")
df_detail.head(22)


df_detail_22 = df_detail[["Close", "Week_Number", "Year"]]
df_detail_22 = df_detail_22[df_detail_22.Year == 2022]
df_22_week = df_detail_22.groupby("Week_Number").last()
df_22_week.head()

,Close,Year
Week_Number,,
1,26.290001,2022
2,26.420000,2022
3,26.580000,2022
4,26.090000,2022
5,25.930000,2022


In [16]:
df_22_week.tail()

,Close,Year
Week_Number,,
48,25.180000,2022
49,25.129999,2022
50,25.299999,2022
51,25.120001,2022
52,25.370001,2022


In [17]:
# buy_and_hold
buy_hold_add = df_22_week.Close.values[-1] - df_22_week.Close.values[0]
buy_hold_add

-0.9200000762939453

In [18]:
# my strategy
def get_benefit_my_strategy(pred_labels, close_prices):
    money = 100
    hold_share = False
    for i in range(len(pred_labels)): # 1 red
#         print(pred_labels[i], close_prices[i])
        
        if (pred_labels[i] == 1): # red---> if not owning, buy one
            if hold_share:
                pass
            else:
                hold_share = True
                money -= close_prices[i]
        else:           # green ---->if owning, sell it
            if hold_share:
                hold_share = False
                money += close_prices[i]
                
        
    share_value = close_prices[-1] if hold_share else 0
    return money + share_value - 100
                    

get_benefit_my_strategy(df_22['predict'].values.tolist(), df_22_week.Close.values.tolist())

1.9299964904785156

In [19]:
# my strategy > buy_and_hold. it is better